# Set run directory

In [74]:
from pathlib import Path
runDir = Path('example.laplace3d_jacobi_2018-04-26_11-34-12')
runDir.exists()

True

# Update records

In [75]:
import pandas as pd
from pathlib import Path
from json import load
from os import stat

# Check for errors
if stat(runDir / 'err.txt').st_size != 0:
    raise ValueError('The error file is not empty! Check err.txt!')
if stat(runDir / 'out.txt').st_size != 0:
    raise ValueError('Something was written to standard output! Check out.txt!')

# Open records
records = pd.DataFrame()
if Path('records.csv').exists():
    records = pd.read_csv('records.csv')

with (runDir / '0.json').open('r') as recF: 
    data = load(recF)
    del data['CV sample times [s]']
    del data['CV samples']
    
    records = records.append(data, ignore_index=True)
records.tail()

,Device,Domain partitions,Exact solution,Executable name,Executable path,Info,Initial guess,Iterant,Matrix,Nodes,...,Processes per node,Rank,Relative error,Relative residual,Threads per process,Timeout [min],Timestamp,Tolerance,Max iterations,Steps
0,julia,"[[0, 100], [0, 100], [0, 100]]",white-noise sine,example.laplace3d_ajacobi,~/asynchronator/build/example,SIAM,zero,ajacobi,laplace3d,1.0,...,1.0,0.0,0.000100,3.500879e-05,256.0,2.0,2018-04-26 11:34:42,0.0001,NaN,NaN
1,julia,"[[0, 25, 50, 75, 100], [0, 50, 100], [0, 50, 1...",white-noise sine,example.laplace3d_ajacobi,~/asynchronator/build/example,SIAM,zero,ajacobi,laplace3d,16.0,...,1.0,0.0,0.000236,2.377630e-07,256.0,2.0,2018-04-26 11:34:54,0.0001,NaN,NaN
2,julia,"[[0, 100], [0, 100], [0, 100]]",white-noise sine,example.laplace3d_jacobi,~/asynchronator/build/example,SIAM,zero,jacobi,laplace3d,1.0,...,1.0,0.0,0.000100,3.244886e-05,256.0,2.0,2018-04-26 11:33:29,0.0001,1000000.0,1.0
3,julia,"[[0, 25, 50, 75, 100], [0, 50, 100], [0, 50, 1...",white-noise sine,example.laplace3d_jacobi,~/asynchronator/build/example,SIAM,zero,jacobi,laplace3d,16.0,...,1.0,0.0,0.000414,3.969562e-05,256.0,2.0,2018-04-26 11:34:12,0.0001,1000000.0,1.0


# Save records

In [76]:
records.to_csv('records.csv', index=False)

# Read iterations record

In [77]:
# Read files
itersData = []
for runP in runDir.glob('*.json'):
    if runP.name == 'setup.json': 
        continue
    with runP.open('r') as recF: 
        data = load(recF)
        dataDf = pd.DataFrame(data['CV samples'], 
                              index=data['CV sample times [s]'],
                              columns =['Rank {}'.format(runP.name[:-5])])
        dataDf.index.name = 'Time [s]'
        itersData.append(dataDf)

itersDf = pd.concat(itersData, axis=1)
itersDf.head()

,Rank 5,Rank 14,Rank 1,Rank 8,Rank 12,Rank 4,Rank 0,Rank 10,Rank 7,Rank 15,Rank 3,Rank 13,Rank 6,Rank 11,Rank 2,Rank 9
Time [s],,,,,,,,,,,,,,,,
0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
0.000003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
0.000003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
0.000003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
0.000004,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Save iterations record

In [78]:
itersDf.to_csv('{}.csv'.format(runDir.name))

# Clean run directory

In [79]:
for f in runDir.glob('*'):
    f.unlink()
runDir.rmdir()